GROUP 10: Thy Bui, Naseem Dabiran, Yifeng Yang 

Narrative of Project: We have decided to web scrape Indeed.com to create a pandas dataframe for Analytics positions in New York City. 
The  DataFrame includes the job position, company, location, company website, company ratings and job salary for each position. 


Note: The Indeed URL is dynamic, so the companies may change with every run

In [1]:
#import libraries
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import requests
import re

import pandas as pd

In [2]:
URL = "https://www.indeed.com/jobs?q=analytics&l=New+York&vjk=d17bf3f8f918cc5e"
r = requests.get(URL) 
c = r.text

soup = BeautifulSoup(c, "html.parser") #getting job postings

In [3]:
#look for td tag and taking attribute resultContent
x = soup.find_all("td", class_="resultContent") 

print(len(x)) #find how many job postings we extracted

# each job posting has a job position, company, and location
# some also include company website, company ratings and job salary
for item in x: 
    print(item) 

15
<td class="resultContent"><div class="heading4 color-text-primary singleLineTitle tapItem-gutter"><h2 class="jobTitle jobTitle-color-purple jobTitle-newJob"><div class="new topLeft holisticNewBlue desktop"><span class="label">new</span></div><a aria-label="full details of Strategy and Data Analyst, Sustainability" class="jcs-JobTitle" data-ci="372618939" data-empn="8589170693800026" data-hide-spinner="true" data-hiring-event="false" data-jk="7f6efc8056f53e09" data-mobtk="1g3lvp8r2grgg800" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0AVIi8UxprrPGU7QPohOxOOpynq0pcPnEidcD-eE3H2ShF7yHgrnn7VX9H7Gcct6uwWAxkHuGso-QvcxjZmB7EadJCXeIEqgnK8JuHAdIZ5cje_hyqn7VefzTE36XVVSFALhZQJ33_gs3ZAFysFnRau86DBL1iKB0TskCCeGRuwTpp9eKbjZfRu-nM8prKHmzmhSRY2sy55qcRUeiimMFxGRO6MrcLbsd3ss8d-XzIg6odyvdAyPIekVd18d-v5PUTO1RGN-YsY6rY_l149NlOF3QckYaSJ3-f_B0vkdlc00ReynaaAFNvH21VMxSqQSwd0CgpOywKzV5hHPsb_lSDwIxAnwdoekOI6FQ1H4jeGtYypi0ww1XrL1_MDvH7mqmK0W8ccPXQ_zw4WggXTuBJBDvneUiG1S9wzBDgqcR5nEmd4Y3615DBqZAYOcl_HGOYO4cldRJua9STly

In [4]:
#Functions to extract information: 

def extract_company(j):
    return j.find("span", class_ = "companyName").get_text()

def extract_position(j):
    return j.find("span", title = re.compile(".*?")).get_text()

def extract_salary(j):
    if j.find("span", class_ = "estimated-salary") is not None:
        return j.find("span", class_ = "estimated-salary").svg["aria-label"]
    # specifed Not provided rather than leave it as None data type
    # to fix error when writing csv file
    else:
        return "Not provided" 

def extract_location(j):
    return j.find("div", class_ = "companyLocation").get_text()

def extract_rating(j):
    if j.find("span", class_ = "ratingsDisplay withRatingLink") is not None: 
        return j.find("span", class_ = "ratingsDisplay withRatingLink").a['aria-label']
    # specifed Not provided rather than leave it as None data type
    # to fix error when writing csv file
    else:
        return "Not provided"
    
def extract_webpage(j):
    if j.find("span", class_="companyName").a is not None: 
        originalLink = "www.indeed.com"
        link = originalLink + j.find("span", class_="companyName").a["href"]
        return link
    else: 
        return "Not provided" 
    

In [5]:
jobs = {} #create empty dictionary

for job in x: 
    
    #call functions to fill dictionary
    
    company = extract_company(job)
    jobs[company] = {} #set company name to empty dictionary
    
    jobs[company]["Position"] = extract_position(job)
    
    jobs[company]["Salary"] = extract_salary(job)  
    
    jobs[company]["Location(s)"] = extract_location(job) 
    
    jobs[company]["Company Indeed Webpage"] =  extract_webpage(job)
    
    jobs[company]["Company Rating"] = extract_rating(job)
    
for k,v in jobs.items(): 
    print(k,v, sep="\n") #print to see elements of dictionary

Uber
{'Position': 'Analytics & Insights Manager, US&C New Verticals', 'Salary': 'Not provided', 'Location(s)': 'New York, NY 10001 (Chelsea area)', 'Company Indeed Webpage': 'www.indeed.com/cmp/Uber', 'Company Rating': 'Company rating 3.7 out of 5 stars'}
LinkIt!
{'Position': 'Data Analyst', 'Salary': 'Not provided', 'Location(s)': 'Manhattan, NY 10011 (Chelsea area)', 'Company Indeed Webpage': 'Not provided', 'Company Rating': 'Not provided'}
Duolingo
{'Position': 'Data Scientist, Analytics', 'Salary': 'Estimated $100K to $127K a year', 'Location(s)': 'New York, NY+1 location', 'Company Indeed Webpage': 'www.indeed.com/cmp/Duolingo', 'Company Rating': 'Company rating 3.9 out of 5 stars'}
Blue-chip Hedge Fund
{'Position': 'Hedge Fund Analyst', 'Salary': 'Not provided', 'Location(s)': 'New York, NY 10022 (Midtown area)', 'Company Indeed Webpage': 'Not provided', 'Company Rating': 'Not provided'}
Delaware North
{'Position': 'Aristocrat Systems Analyst Gaming, Remote', 'Salary': 'Not prov

In [6]:
#Construct Pandas DataFrame from dictionary 
jobs_df = pd.DataFrame(jobs)
jobs_df.head()

,Uber,LinkIt!,Duolingo,Blue-chip Hedge Fund,Delaware North,Spotify,TIAA,New York University,Disney Streaming,US Federal Aviation Administration,PCS GLOBAL TECH,KPMG,Nike,Hudson Valley Environmental Solutions
Position,"Analytics & Insights Manager, US&C New Verticals",Data Analyst,"Data Scientist, Analytics",Hedge Fund Analyst,"Aristocrat Systems Analyst Gaming, Remote","Analytics Engineer, Talk",Analyst – Nuveen Global Impact,PC Support Technician III,QA Analyst I,Management and Program Analyst,Data analyst/ Entry Level position/ SQL Develo...,"Associate, Data & Analytics",Principal Product Director – s23NYC (Open to r...,Water Analyst - Bilingual A Plus
Salary,Not provided,Not provided,Estimated $100K to $127K a year,Not provided,Not provided,Estimated $68.8K to $87.2K a year,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided
Location(s),"New York, NY 10001 (Chelsea area)","Manhattan, NY 10011 (Chelsea area)","New York, NY+1 location","New York, NY 10022 (Midtown area)","Remote in Buffalo, NY 14202","Remote in New York, NY+1 location","New York, NY","New York, NY 10012 (Greenwich Village area)","New York, NY","Queens, NY","New York, NY","Melville, NY 11747+3 locations","Remote in New York, NY","Newburgh, NY 12550"
Company Indeed Webpage,www.indeed.com/cmp/Uber,Not provided,www.indeed.com/cmp/Duolingo,Not provided,www.indeed.com/cmp/Delaware-North,www.indeed.com/cmp/Spotify,www.indeed.com/cmp/Tiaa,www.indeed.com/cmp/New-York-University,Not provided,www.indeed.com/cmp/Federal-Aviation-Administra...,Not provided,www.indeed.com/cmp/Kpmg-0828bc85,www.indeed.com/cmp/Nike,Not provided
Company Rating,Company rating 3.7 out of 5 stars,Not provided,Company rating 3.9 out of 5 stars,Not provided,Company rating 3.7 out of 5 stars,Company rating 4.3 out of 5 stars,Company rating 3.7 out of 5 stars,Company rating 4.2 out of 5 stars,Not provided,Company rating 4.1 out of 5 stars,Not provided,Company rating 3.9 out of 5 stars,Company rating 4.1 out of 5 stars,Not provided


In [7]:
import json

#create new json file and copy content of jobs dictionary
with open("JobSearch.json", "w") as writeJSON: 
    json.dump(jobs, writeJSON)


In [8]:
import csv
   
#create new csv file and copy contents of jobs dictionary
with open("JobSearch.csv", "w") as toWrite:  
    writer = csv.writer(toWrite, delimiter = ",")
    writer.writerow(["Company","Position","Salary","Location(s)", "Company Indeed Webpage", "Company Rating"])
    for a in jobs.keys():
        writer.writerow([a, jobs[a]["Position"], jobs[a]["Salary"], jobs[a]["Location(s)"], jobs[a]["Company Indeed Webpage"], jobs[a]["Company Rating"]])
        
